In [1]:
import os

os.environ["CONFIG_APP_DIR"] = "config"


In [2]:
import sys
import os
import pickle
os.chdir("..")
from omegaconf import OmegaConf

import json
from etl_pipeline.data_processor_engine.json_engine.json_engine import JsonProcessingEngine
from etl_pipeline.custom.ms.payload_loader import PayloadLoader

d:\ProgramData\Anaconda3\envs\etl\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
from etl_pipeline.config import pipeline_config
cn = pipeline_config.cn

In [4]:
# alert = load_alert()

In [5]:
# payload = load_alert()


# Definition

In [6]:
from pipelines.ms.ms_pipeline import MSPipeline
from etl_pipeline.config import pipeline_config


In [25]:
with open(f'notebooks/sample/wm_address_in_payload_format.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)



payload_json['match_ids'] = [i for i in range(len(payload_json["watchlistParty"]["matchRecords"]))]


In [ ]:
payload_json

In [13]:
payload = payload_json

In [14]:
from etl_pipeline.config import load_agent_configs, pipeline_config

In [15]:
engine = JsonProcessingEngine(pipeline_config)
pipeline = MSPipeline(engine, config=pipeline_config)

Field in agent configuration for: AP_PARTY_TYPE is not registered field in pipeline.yaml


## transform standardized to cleansed

In [16]:
payload = pipeline.transform_standardized_to_cleansed(payload)

> d:\repos\etl-pipeline\etl_pipeline\data_processor_engine\json_engine\json_engine.py(270)collect_party_values_from_parties_from_fields()
    268             if connected_key == "ALL_CONCAT_NAMES":
    269                 import pdb; pdb.set_trace()
--> 270             payload[connected_key] = self.get_field_value_name(fields, value)
    271 
    272     def get_field_value_name(self, fields, name):

ipdb> self.get_field_value_name(fields, value)
'Joe BlackJ'
ipdb> c
> d:\repos\etl-pipeline\etl_pipeline\data_processor_engine\json_engine\json_engine.py(270)collect_party_values_from_parties_from_fields()
    268             if connected_key == "ALL_CONCAT_NAMES":
    269                 import pdb; pdb.set_trace()
--> 270             payload[connected_key] = self.get_field_value_name(fields, value)
    271 
    272     def get_field_value_name(self, fields, name):

ipdb> c


ALL_CONNECTED_ACCOUNT_NAMES# Transform standardized to application

In [17]:
new_payloads = pipeline.transform_cleansed_to_application(payload)

No field in payload named: hit_type_agent_country.
No field in payload named: hit_type_agent_job.
No field in payload named: hit_type_agent_other.
No field in payload named: hit_type_agent_country.
No field in payload named: hit_type_agent_job.
No field in payload named: hit_type_agent_other.
No field in payload named: hit_type_agent_country.
No field in payload named: hit_type_agent_job.
No field in payload named: hit_type_agent_other.
No field in payload named: hit_type_agent_country.
No field in payload named: hit_type_agent_job.
No field in payload named: hit_type_agent_other.


### payload

In [18]:
len(new_payloads)

2

In [19]:
new_payloads

[{'alertedParty': {'headerInfo': {'currentVersionId': '122438658',
    'datasetId': '1044',
    'datasetName': 'R_US_Active_Address',
    'firstVersionCreatedDt': '2020-01-08T09:31:30.264-05:00',
    'inputVersionSample': 'R_US_Active_Address_A05003324172_2020-01-07-07.06.',
    'lastVersionUpdatedDt': '2020-01-08T09:31:30.264-05:00',
    'masterId': '72951854',
    'masterVersion': '412740c151535f0756e8dbec7440b726c7a3e135',
    'stopDescriptors': {'stopDescriptor': {'name': 'Joe Doe'}},
    'uniqueCustomerId': 'R_US_Active_Address_A05003324172_2020-01-07-07.06.28.836480'},
   'inputRecordHist': {'inputRecords': [{'createdDate': '01/08/20',
      'fields': [{'isScreenable': 'false',
        'name': 'SOURCE_REF',
        'sortOrder': '1',
        'value': 'R_US_Active_Address_A05003324172_2020-01-07-07.06.28.836480'},
       {'isScreenable': 'false',
        'name': 'UNIQUE_KEY',
        'sortOrder': '2',
        'value': 'A05003324172'},
       {'isScreenable': 'false', 'name': 'BARCO

In [21]:
import pandas as pd
out_payload = pd.DataFrame([match for payload in new_payloads   for match in payload['watchlistParty']['matchRecords']])

out_payload['AP_TRIGGERS'].values[0]

{'Doe': {'ADDRESS1_LINE2': ['Jane Doe AA BBB'],
  'CONCAT_ADDRESS': ['Joe Black & Jane Doe 111 A 11TH ST AAA 1A New Jersey NJ']}}

In [22]:
out_payload

,accountSeq,datasetId,entity,entityId,entityTextType,entityType,entityVersion,firstMatchedDate,inputVersionId,lastMatchedDate,...,ap_all_ap_id_tp_markeds_aggregated,wl_all_ap_id_tp_markeds_aggregated,ap_all_is_entity_matchs_aggregated,wl_all_is_entity_matchs_aggregated,all_hit_type_aggregated,WL_ADDRESS1,WL_ADDRESS2,WL_CITY,WL_COUNTRYNAME,WL_POSTALCODE
0,1,1044,"{'addresses': [{'address1': '111 STREET 999', ...",908043,UPID,03,20150505194929,01/08/20,122438658,01/08/20,...,[],[],[I],[I],"[{'name': ['PARTY1_NAME_FULL', 'CONCAT_NAME'],...",NaN,NaN,NaN,NaN,NaN
1,1,1044,{'addresses': {'address': {'address1': '111 ST...,908044,UPID,03,20150505194923,01/08/20,122438659,01/08/20,...,[],[],[I],[I],"[{'name': [], 'country': ['ADDRESS1_COUNTRY'],...",[111 STREET 999],[FL;US],[ATLANTA],[Poland],[11111]


In [24]:
out_payload[[i for i in out_payload.columns if "name" in i ]]

,company_name_agent_ap,company_name_agent_wl,name_agent_ap,name_agent_wl,employer_name_agent_ap,employer_name_agent_wl,hit_type_agent_name,ap_name_tp_marked_agent_input,ap_all_company_names_aggregated,wl_all_company_names_aggregated,ap_all_names_aggregated,wl_all_names_aggregated,ap_all_employer_names_aggregated,wl_all_employer_names_aggregated,ap_all_ap_name_tp_markeds_aggregated,wl_all_ap_name_tp_markeds_aggregated
0,"[Joe BlackJ, [Pladimir Vutin, Eva Pladimirova]...",Joe Ding,"[Joe BlackJ, [Pladimir Vutin, Eva Pladimirova]...",Joe Ding,"[None, None]","[Joe Ding, []]","[PARTY1_NAME_FULL, CONCAT_NAME]",Janusz,"[Joe BlackJ, Eva Pladimirova, Janusz, Pladimir...",[Joe Ding],"[Joe BlackJ, Eva Pladimirova, Janusz, Pladimir...",[Joe Ding],[],[Joe Ding],[],[]
1,"[None, [Pladimir Vutin, Eva Pladimirova], [Pla...",Joe Dong,"[None, [Pladimir Vutin, Eva Pladimirova], [Pla...",Joe Dong,"[None, None]","[Joe Dong, []]",[],None,"[Eva Pladimirova, Pladimir Vutin]",[Joe Dong],"[Eva Pladimirova, Pladimir Vutin]",[Joe Dong],[],[Joe Dong],[],[]


In [ ]:
for payload in new_payloads:
    print('=====================')
    print('PAYLOAD')
    print(payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        
    for num, match in enumerate(payload['watchlistParty']['matchRecords']):
        assert (match['inputVersionId'] == payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        print(num, 'match')
        print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
        try:
            print(match["ap_all_residencies_aggregated"], match["wl_all_residencies_aggregated"])
        except:
            break


In [ ]:
out_payload[[i for i in out_payload.columns if "hit" in i and "agg" in i]]

In [ ]:
import pickle
with open("tests/shared/parsed_payload.pkl", "wb") as f:
    pickle.dump(new_payloads, f)

In [ ]:
# import pickle
# with open("tests/shared/parsed_payload.pkl", "rb") as f:
#     p = pickle.load(f)

# import pandas as pd
# reference_payload = pd.DataFrame([match for payload in p  for match in payload['watchlistParty']['matchRecords']])



In [ ]:
out_payload['TRIGGERED_BY']  = out_payload['TRIGGERED_BY'].apply(sorted)
reference_payload['TRIGGERED_BY']  = reference_payload['TRIGGERED_BY'].apply(sorted)


In [ ]:
out_payload['AP_TRIGGERS']  = out_payload['AP_TRIGGERS'].apply(lambda x: {key: x[key] for key in sorted(x)})
reference_payload['AP_TRIGGERS']  = reference_payload['AP_TRIGGERS'].apply(lambda x: {key: x[key] for key in sorted(x)})


In [ ]:
for cols in out_payload.columns:
    try:
        pd.testing.assert_series_equal(out_payload[cols], reference_payload[cols])
    except (AssertionError, TypeError):
        reference = remove_nulls_from_aggegated(flatten([i for i in reference_payload[cols].values]))
        output = remove_nulls_from_aggegated(flatten([i for i in out_payload[cols].values]))
        try:
            if isinstance(output, list):
                assert_list(output, reference)
            else:
                assert output == reference
        except AssertionError:
            print((cols, output, reference))
            pass

In [ ]:
while True:
    print("remove", columns[-1])
    columns = columns[:-1]
    try:
        pd.testing.assert_frame_equal(out_payload[columns], reference_payload[columns])
        print('ok')
        break
    except AssertionError:
        break
        pass
    except TypeError:
        pass

In [ ]:
out_payload[[i for i in out_payload.columns if "hit" in i and "agg" in i]]

In [ ]:
with open(f'notebooks/sample/wm_address_in_payload_format_2_input_3_match_records.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)



payload_json['match_ids'] = [i for i in range(len(payload_json["watchlistParty"]["matchRecords"]))]


In [ ]:
payload = payload_json

In [ ]:
payload = pipeline.transform_standardized_to_cleansed(payload)

In [ ]:
new_payloads = pipeline.transform_cleansed_to_application(payload)

In [ ]:
import pandas as pd
out_payload = pd.DataFrame([match for payload in new_payloads   for match in payload['watchlistParty']['matchRecords']])

out_payload[[i for i in out_payload.columns  if i.endswith("_aggregated")]]

In [ ]:
for payload in new_payloads:
    print('=====================')
    print('PAYLOAD')
    print(payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        
    for num, match in enumerate(payload['watchlistParty']['matchRecords']):
        assert (match['inputVersionId'] == payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        print(num, 'match')
        print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
        try:
            print(match["ap_all_residencies_aggregated"], match["wl_all_residencies_aggregated"])
        except:
            break


In [ ]:
import pickle
with open("tests/shared/parsed_payload_2_payload.pkl", "wb") as f:
    pickle.dump(new_payloads, f)

In [ ]:
new_dict = {}
def return_key(dict_, prefix):
    
    if isinstance(dict_, list):
        for num, i in enumerate(dict_):
            return_key(i, prefix +f"[{num}]")
        return
    if isinstance(dict_, str) or dict_ is None:
        new_dict[prefix] = dict_
        return

    for key in dict_:
        if prefix:
            basic_prefix = prefix + "." + key
        else:
            basic_prefix = key
        return_key(dict_[key], basic_prefix)

In [ ]:
with open(f'notebooks/sample/wm_party_payload_without_supplemental_info.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)



payload_json['match_ids'] = [i for i in range(len(payload_json["watchlistParty"]["matchRecords"]))]


In [ ]:
payload = payload_json

In [ ]:
payload = pipeline.transform_standardized_to_cleansed(payload)

In [ ]:
new_payloads = pipeline.transform_cleansed_to_application(payload)

In [ ]:
import pandas as pd
out_payload = pd.DataFrame([match for payload in new_payloads   for match in payload['watchlistParty']['matchRecords']])

out_payload[[i for i in out_payload.columns  if i.endswith("_aggregated")]]

In [ ]:
for payload in new_payloads:
    print('=====================')
    print('PAYLOAD')
    print(payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        
    for num, match in enumerate(payload['watchlistParty']['matchRecords']):
        assert (match['inputVersionId'] == payload['alertedParty']['inputRecordHist']['inputRecords'][0]['versionId'])
        print(num, 'match')
        print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
        try:
            print(match["ap_all_residencies_aggregated"], match["wl_all_residencies_aggregated"])
        except:
            break


In [ ]:
import pickle
with open("tests/shared/empty_payload.pkl", "wb") as f:
    pickle.dump(new_payloads, f)

In [ ]:
from etl_pipeline.config import load_agent_config

In [ ]:
import pandas as pd
out_payload = pd.DataFrame([match for payload in new_payloads   for match in payload['watchlistParty']['matchRecords']])

out_payload[[i for i in out_payload.columns  if i.endswith("_aggregated")]]

In [ ]:
from omegaconf import OmegaConf
alert_agents_config = {
        "alert_type": OmegaConf.load(os.path.join(CONFIG_APP_DIR, "agents", "agents.yaml"))
    }


import omegaconf
def get_fields(dict_):
    fields = []
    for key in dict_:
        
        if isinstance(dict_[key], omegaconf.dictconfig.DictConfig):
            new_fields = get_fields(dict_[key])
            fields.extend(new_fields)
        else:
#             import pdb; pdb.set_trace()
            fields.extend(dict_[key])
    return fields

fields = [i.split('.')[-1] for i in get_fields(alert_agents_config['alert_type']) if "alertedParty.inputRecordHist.inputRecords.INPUT_FIELD" not in i]
    
    

from glob import glob

files = glob("pipelines/**/*.py",  recursive=True)
files.extend(glob("etl_pipeline/**/*.py",  recursive=True))

fields_to_check = list(fields)
for file in files:
    with open(file) as f:
        text = f.read()
    new_fields_to_check = []
    
    for field in fields_to_check:
        if field in text:
            continue
        new_fields_to_check.append(field)
    fields_to_check = new_fields_to_check
        
    

fields_to_check

CONFIG_APP_DIR = os.environ["CONFIG_APP_DIR"] 

In [ ]:
import pickle

In [ ]:
with open("/tmp/request.pkl", 'rb') as f:
    r = pickle.load(f)

In [ ]:
from silenteight.datasource.api.name.v1.name_pb2 import (
    NameFeatureInput,
)

In [ ]:
from google.protobuf.any_pb2 import Any

In [ ]:
some_any2 = Any()
message = NameFeatureInput()
some_any2.Pack(NameFeatureInput())
# some_any2.CopyFrom(i.feature_inputs[0].agent_feature_input)


In [ ]:
i.feature_inputs[0].agent_feature_input

In [ ]:
i.feature_inputs[0].agent_feature_input.Unpack(message)

In [ ]:
message

In [ ]:
unpacked_msg = extension_pb2.GeneralContent_Extension())


In [ ]:
NameFeatureInput.ParseFromString()

In [ ]:
from sile
i.feature_inputs[0].agent_feature_input.value.decode('utf-8')

In [ ]:
type(i.feature_inputs[0].agent_feature_input.type_url)

In [ ]:
for i in response.agent_inputs:
    print(i.feature_inputs)

In [ ]:
for 